In [1]:
import pandas as pd
import shapely
import numpy as np
import sys
import argparse
import random
import math
import os
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import stats
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 999
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

In [6]:
# read file
data = pd.read_pickle('../results/new_sample.tmp')

In [9]:
data.head()

,Jobid,Rot,List of Coordinates,Shapely Polygon,Metric1,Metric2,Metric3,Metric4,Area,Length,Left down,score,right down
16,92600-1250_7_16,0.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",17.00,132.00,849.00,5.00,4439.800,615.763,"(256.8, 849.0)",0.500,"(256.8, 903.5)"
17,92600-1250_7_17,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",18.00,288.00,1354.00,4.00,4439.800,615.763,"(342.5, 1354.0)",4.000,"(342.5, 1478.8)"
18,92600-1250_7_18,90.00,"[(1.0, 123.3), (0.0, 122.3), (0.0, 93.3), (1.0...","POLYGON ((1 123.3, 0 122.3, 0 93.3, 1 92.3, 21...",19.00,386.00,1352.00,4.00,7228.382,901.580,"(509.3, 1352.0)",4.000,"(509.3, 1455.6)"
19,92600-1250_7_19,90.00,"[(6.0, 38.0), (7.1, 40.0), (97.5, 40.0), (98.7...","POLYGON ((6 38, 7.1 40, 97.5 40, 98.7 38, 98.7...",20.00,135.00,1186.00,2.00,23178.165,1448.933,"(175.0, 1186.0)",0.000,"(175.0, 1874.3)"
20,92600-1250_7_20,90.00,"[(28.5, 0.0), (60.1, 0.0), (84.4, 7.9), (88.3,...","POLYGON ((28.5 0, 60.1 0, 84.40000000000001 7....",21.00,491.00,1466.00,4.00,1103.760,192.719,"(511.0, 1466.0)",4.000,"(511.0, 1554.3)"


In [12]:
def bin_6(x):
    binofx = bin(x)[2:]
    str_ = ""
    if len(binofx) < 6:
        return str_.join((str_.join(["0"]*(6-len(binofx))),binofx))
    else:
        return binofx

In [13]:
data['seq_in_job'] = data['Jobid'].map(lambda x: x.split('_')[-1]).astype(int)
data['bin_seq_in_job'] = data['seq_in_job'].map(lambda x: bin_6(x))
data["batch_jobid"] =  data['Jobid'].map(lambda x: x.split('_')[0]+"_"+x.split('_')[1])
data['num_of_corr'] = data['List of Coordinates'].map(lambda x: len(x)).astype("category")

In [14]:
tmp = pd.get_dummies(data["num_of_corr"])
tmp.columns = ["13_coords", "17_coords","18_coords","23_coords","26_coords","54_coords"]
data = pd.concat([data, tmp], axis=1)

In [15]:
data['Rot'][data['Rot'] == '5.00'] = '0.00'
tmp = pd.get_dummies(data["Rot"])
tmp.columns = ["Rot_0", "Rot_90"]
data = pd.concat([data, tmp], axis=1)

In [16]:
temp = "".join(list(data["bin_seq_in_job"].values))
array_temp = np.array([int(i) for i in temp]).reshape(-1,6)
DF_temp = pd.DataFrame(array_temp, columns = ["pos_0", "pos_1", "pos_2", "pos_3", "pos_4", "pos_5"])
DF_temp.index= data.index
data = pd.concat([data, DF_temp], axis=1)

In [17]:
data

,Jobid,Rot,List of Coordinates,Shapely Polygon,Metric1,Metric2,Metric3,Metric4,Area,Length,Left down,score,right down,seq_in_job,bin_seq_in_job,batch_jobid,num_of_corr,13_coords,17_coords,18_coords,23_coords,26_coords,54_coords,Rot_0,Rot_90,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5
16,92600-1250_7_16,0.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",17.00,132.00,849.00,5.00,4439.800,615.763,"(256.8, 849.0)",0.500,"(256.8, 903.5)",16,010000,92600-1250_7,18,0,0,1,0,0,0,1,0,0,1,0,0,0,0
17,92600-1250_7_17,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",18.00,288.00,1354.00,4.00,4439.800,615.763,"(342.5, 1354.0)",4.000,"(342.5, 1478.8)",17,010001,92600-1250_7,18,0,0,1,0,0,0,0,1,0,1,0,0,0,1
18,92600-1250_7_18,90.00,"[(1.0, 123.3), (0.0, 122.3), (0.0, 93.3), (1.0...","POLYGON ((1 123.3, 0 122.3, 0 93.3, 1 92.3, 21...",19.00,386.00,1352.00,4.00,7228.382,901.580,"(509.3, 1352.0)",4.000,"(509.3, 1455.6)",18,010010,92600-1250_7,54,0,0,0,0,0,1,0,1,0,1,0,0,1,0
19,92600-1250_7_19,90.00,"[(6.0, 38.0), (7.1, 40.0), (97.5, 40.0), (98.7...","POLYGON ((6 38, 7.1 40, 97.5 40, 98.7 38, 98.7...",20.00,135.00,1186.00,2.00,23178.165,1448.933,"(175.0, 1186.0)",0.000,"(175.0, 1874.3)",19,010011,92600-1250_7,17,0,1,0,0,0,0,0,1,0,1,0,0,1,1
20,92600-1250_7_20,90.00,"[(28.5, 0.0), (60.1, 0.0), (84.4, 7.9), (88.3,...","POLYGON ((28.5 0, 60.1 0, 84.40000000000001 7....",21.00,491.00,1466.00,4.00,1103.760,192.719,"(511.0, 1466.0)",4.000,"(511.0, 1554.3)",20,010100,92600-1250_7,13,1,0,0,0,0,0,0,1,0,1,0,1,0,0
21,92600-1250_7_21,0.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",22.00,467.00,1231.00,5.00,4439.800,615.763,"(591.8, 1231.0)",0.500,"(591.8, 1285.5)",21,010101,92600-1250_7,18,0,0,1,0,0,0,1,0,0,1,0,1,0,1
22,92600-1250_7_22,90.00,"[(1.0, 123.3), (0.0, 122.3), (0.0, 93.3), (1.0...","POLYGON ((1 123.3, 0 122.3, 0 93.3, 1 92.3, 21...",23.00,497.00,1299.00,2.00,7228.382,901.580,"(620.3, 1299.0)",0.000,"(620.3, 1402.6)",22,010110,92600-1250_7,54,0,0,0,0,0,1,0,1,0,1,0,1,1,0
23,92600-1250_7_23,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",24.00,584.00,1433.00,2.00,4439.800,615.763,"(638.5, 1433.0)",0.000,"(638.5, 1557.8)",23,010111,92600-1250_7,18,0,0,1,0,0,0,0,1,0,1,0,1,1,1
24,92600-1250_7_24,0.00,"[(0.0, 50.6), (0.0, 114.3), (3.0, 117.3), (42....","POLYGON ((0 50.6, 0 114.3, 3 117.3, 42 117.3, ...",25.00,186.00,87.00,2.00,25195.570,1434.169,"(794.5, 87.0)",0.000,"(794.5, 204.3)",24,011000,92600-1250_7,23,0,0,0,1,0,0,1,0,0,1,1,0,0,0
25,92600-1250_7_25,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",26.00,528.00,1238.00,2.00,4439.800,615.763,"(582.5, 1238.0)",0.000,"(582.5, 1362.8)",25,011001,92600-1250_7,18,0,0,1,0,0,0,0,1,0,1,1,0,0,1


In [18]:
result = []
for name, group in data.groupby('batch_jobid'):
    if len(group) == 50:
        temp_x = group[["13_coords","17_coords","18_coords","23_coords","26_coords","54_coords","Rot_0","Rot_90",
                           "pos_0", "pos_1", "pos_2", "pos_3", "pos_4", "pos_5"]].values.reshape(1,-1)
        # ------------------------------------------------------
        mean = group["score"].mean()
        max_x = np.max([i[0] for i in group["Left down"].values])
        max_y = np.max([i[1] for i in group["right down"].values])
        area_quote = 1-group["Area"].sum()/(max_x*max_y)
        temp_df = pd.DataFrame(np.concatenate((temp_x,[mean, max_x, area_quote]), axis=None).reshape(1,-1), index = [name])
        # --------------------------------------------------------
        result.append(temp_df)

In [19]:
result = pd.concat(result)

In [20]:
result.to_csv("regression.csv")

In [21]:
result.iloc[:,701].describe()

count   15302.000
mean     1879.890
std       325.726
min       935.500
25%      1637.000
50%      1853.300
75%      2099.500
max      3037.500
Name: 701, dtype: float64